In [4]:
!pip install pinecone
!pip install google-generativeai
!pip install langchain
!pip install langchain_community
!pip install langchain-google-genai
!pip install google-generativeai
!pip install PyPDF2
!pip install -U google-generativeai langchain-google-genai
!pip install pypdf

  Using cached pinecone-7.2.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.2.0-py3-none-any.whl (524 kB)
  Using cached langchain_google_genai-2.1.5-py3-none-any.whl.metadata (5.2 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached langchain_google_genai-2.1.3-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_google_genai-2.1.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_google_genai-2.1.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_google_genai-2.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached langchain_google_genai-2.0.11-py3-none-any.whl.metadata (3.6 kB)


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Or "gemini-pro"
    google_api_key=""
)

Test the llm

In [6]:
llm.invoke("hello there")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--8eac491e-0bd9-41bc-a359-adeae6124a90-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [7]:
from langchain.document_loaders import DirectoryLoader,TextLoader,PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs")

In [8]:
docs=loader.load()

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
texts=splitter.split_documents(documents=docs)

In [10]:
from pinecone import Pinecone
PINECONE_API_KEY=""
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("shanksi-ndex")

In [14]:
# Prepare the vectors with metadata
final_vecs = [
    {
        "id": f"doc{i+1}",
        "values": vectors[i],
        "metadata": {
            "chunk_text": texts[i]  # Store the original text with each vector
        }
    }
    for i in range(len(vectors))
]

# Upsert into the index
index.upsert(vectors=final_vecs)


{'upserted_count': 30}

In [34]:

query_text = "work experience of Yunlong Jiao"
query_vector = embeddings.embed_query(query_text)

results = index.query(
    vector=query_vector,
    top_k=3,
    include_metadata=True
)

print(llm.invoke(f"answer this {query_text} from this resource : {results['matches']}"))

content="Based on the provided text, Yunlong Jiao's work experience is as follows:\n\n* **Machine Learning Scientist (Current, from Nov 2019): Amazon, London, UK**\n\n    *  At the Alexa Shopping Science team (from March 2021), his responsibilities include:\n        * Leading research on bias mitigation technologies in NLP to enhance fairness while preserving user privacy.\n        * Building and deploying ML solutions to create unified user experiences for Alexa Shopping across different devices.\n    *  Prior to this specific role within the Alexa Shopping Science team, he worked on building AI assistants at Amazon Alexa that replicate human intelligence.  He was involved in all phases of development, from proof of concept to delivering minimum viable products and scaling up production.\n\n\n  The text also mentions that he has experience supervising master's theses and research internships, as well as coaching and mentoring junior team members.  However, these are presented as skill